## Importing

In [1]:
# For data analysis
import pandas as pd
import numpy as np
import datetime

# Importing customized functions 
from customized_packages.customized_functions import get_full_columns, get_numerical_columns, get_categorical_columns,\
                                                    contruct_missing_values_table, display_distribution_of_categorical_values, \
                                                    fill_missing_values, drop_duplicate, \
                                                    get_feature_names, save_prepared_data

# sklearn API to transform data
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

                                                            
# to save downloaded data
import pickle
import os
import warnings
warnings.filterwarnings("ignore")

## Load data

In [2]:
DATA_PATH = "raw_data/bank-full.csv"
raw_bank_df = pd.read_csv(DATA_PATH, sep= ";")

In [3]:
# Get column names
correct_order_columns = get_full_columns()
# Create a backup 
bank_df = raw_bank_df[correct_order_columns].copy()

### Get numerical and categorical columns

In [4]:
numerical_columns = get_numerical_columns()
categorical_columns = get_categorical_columns()

In [5]:
bank_df[numerical_columns].head()

,age,balance,day,duration,campaign,pdays,previous
0,58,2143,5,261,1,-1,0
1,44,29,5,151,1,-1,0
2,33,2,5,76,1,-1,0
3,47,1506,5,92,1,-1,0
4,33,1,5,198,1,-1,0


In [6]:
bank_df[categorical_columns].head()

,job,marital,education,default,housing,loan,contact,month,poutcome
0,management,married,tertiary,no,yes,no,unknown,may,unknown
1,technician,single,secondary,no,yes,no,unknown,may,unknown
2,entrepreneur,married,secondary,no,yes,yes,unknown,may,unknown
3,blue-collar,married,unknown,no,yes,no,unknown,may,unknown
4,unknown,single,unknown,no,no,no,unknown,may,unknown


## Map target variables

In [7]:
mapped = {"yes":1, "no":0}
bank_df["y"] = bank_df["y"].map(mapped)

In [8]:
# Seperate data and labels 
bank = bank_df.drop("y", axis = 1)
bank_labels = bank_df["y"]

## Missing and duplicated values

### Missing values

In [9]:
contruct_missing_values_table(bank_df)

,missing values,percentage of total values
age,0,0.0
job,0,0.0
marital,0,0.0
education,0,0.0
default,0,0.0
housing,0,0.0
loan,0,0.0
contact,0,0.0
day,0,0.0
month,0,0.0


In [10]:
display_distribution_of_categorical_values(bank_df)

col_0,job,percentage of values
0,admin.,0.114375
1,blue-collar,0.215257
2,entrepreneur,0.032890
3,housemaid,0.027427
4,management,0.209197
5,retired,0.050076
6,self-employed,0.034925
7,services,0.091880
8,student,0.020747
9,technician,0.168034


col_0,marital,percentage of values
0,divorced,0.115171
1,married,0.601933
2,single,0.282896


col_0,education,percentage of values
0,primary,0.151534
1,secondary,0.513194
2,tertiary,0.294198
3,unknown,0.041074


col_0,default,percentage of values
0,no,0.981973
1,yes,0.018027


col_0,housing,percentage of values
0,no,0.444162
1,yes,0.555838


col_0,loan,percentage of values
0,no,0.839774
1,yes,0.160226


col_0,contact,percentage of values
0,cellular,0.647741
1,telephone,0.064276
2,unknown,0.287983


col_0,month,percentage of values
0,apr,0.064851
1,aug,0.138174
2,dec,0.004733
3,feb,0.058592
4,jan,0.031032
5,jul,0.152507
6,jun,0.118135
7,mar,0.010551
8,may,0.304483
9,nov,0.087810


col_0,poutcome,percentage of values
0,failure,0.108403
1,other,0.040698
2,success,0.033421
3,unknown,0.817478


> **The missing values appeared in the form of "unknown" values**

In [11]:
# Fill mising values
fill_missing_columns = ["job", "education"]
bank_filled = fill_missing_values(bank, fill_missing_columns = fill_missing_columns)　# using "mode" values

In [12]:
display_distribution_of_categorical_values(bank_filled[fill_missing_columns])

col_0,job,percentage of values
0,admin.,0.114375
1,blue-collar,0.221627
2,entrepreneur,0.032890
3,housemaid,0.027427
4,management,0.209197
5,retired,0.050076
6,self-employed,0.034925
7,services,0.091880
8,student,0.020747
9,technician,0.168034


col_0,education,percentage of values
0,primary,0.151534
1,secondary,0.554268
2,tertiary,0.294198


### Duplicate data

In [13]:
bank_filled = drop_duplicate(bank_filled)

Data shape before replacing duplicated values: (45211, 16)
Data shape after replacing duplicated values: (45211, 16)


## Seperate training and testing data

> **In order to avoid data leak, it is better to seperate training and testing data before any transformation**

In [14]:
training_data, testing_data, training_label, testing_label = train_test_split(bank_filled, bank_labels, test_size = 0.2, random_state=10)

## Build piple line for feature engineering and processing data before training

### Deal with numerical attributes

In [15]:
from customized_packages.customized_transformer import NumericalAttributesTransformer, CustomizedStandardScaler

In [16]:
# Create pipeline to transform numerical values
numerical_attributes = ['balance', 'day', 'duration', 'campaign', 'pdays', 'previous']

to_binary_attribute = "pdays"
# the following attributes will transform to normal distribution
to_log_attributes = ['duration', 'campaign', 'previous']

numerical_transformer = Pipeline([
    ("numerical_transformer", NumericalAttributesTransformer(numerical_attributes, to_binary_attribute, to_log_attributes)),
    ("scaler", CustomizedStandardScaler()),
    ])

### Deal with oridinal and categorical attributes

In [17]:
from customized_packages.customized_transformer import CategoricalAttributesTransformer

In [18]:
categorial_attributes = get_categorical_columns()
oridinal_attribute = "age"
# "age" is numerical values, but it seems to be able to divide into small ranges -> "age" can be treated as oridinal attributes

categorical_transformer = CategoricalAttributesTransformer(categorial_attributes, oridinal_attribute)

## Full pipeline for pre-processing data

In [19]:
numerical_attributes = ['balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
categorical_attributes = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome',"age"]

full_pipeline = ColumnTransformer([
        ("num_pipeline", numerical_transformer, numerical_attributes),
        ("cat_pipeline", categorical_transformer, categorical_attributes),
    ])

In [20]:
# Transforming
bank_training_prepared = full_pipeline.fit_transform(training_data)
bank_testing_prepared = full_pipeline.fit_transform(testing_data)

In [21]:
print(f"The shape of training data: {bank_training_prepared.shape}")
print(f"The shape of testing data: {bank_testing_prepared.shape}")

The shape of training data: (36168, 47)
The shape of testing data: (9043, 47)


In [22]:
# Get back the names of all attributes after transformation
feature_names_after_transform = get_feature_names(numerical_transformer, categorical_transformer, 
                                                numerical_attributes, categorical_attributes, bank_filled)

In [23]:
len(feature_names_after_transform)

47

In [24]:
feature_names_after_transform

Index(['balance', 'day', 'duration', 'campaign', 'previous', 'pdays_binary',
       'job_blue-collar', 'job_entrepreneur', 'job_housemaid',
       'job_management', 'job_retired', 'job_self-employed', 'job_services',
       'job_student', 'job_technician', 'job_unemployed', 'marital_married',
       'marital_single', 'education_secondary', 'education_tertiary',
       'default_yes', 'housing_yes', 'loan_yes', 'contact_telephone',
       'contact_unknown', 'month_aug', 'month_dec', 'month_feb', 'month_jan',
       'month_jul', 'month_jun', 'month_mar', 'month_may', 'month_nov',
       'month_oct', 'month_sep', 'poutcome_other', 'poutcome_success',
       'poutcome_unknown', 'age_group_20-29', 'age_group_30-39',
       'age_group_40-49', 'age_group_50-59', 'age_group_60-69',
       'age_group_70-79', 'age_group_80-89', 'age_group_90-100'],
      dtype='object')

## Save prepared data

In [28]:
# We will save training and testing data with labels and all the features' name after apply transformation
save_data_list = [bank_training_prepared, bank_testing_prepared, training_label, testing_label]
save_prepared_data(save_data_list, 
                   feature_names= feature_names_after_transform, 
                   save_column_names=True)